In [23]:
import math
import numpy as np
import pandas as pd
import pdb

In [24]:
# df = pd.read_csv('pima_redux.tsv', sep = '\t')

In [25]:
# df.values[-1]

In [26]:
class Connection:
    def __init__(self, connectedNeuron, weight=None):
        self.connectedNeuron = connectedNeuron
        
        if weight:
            self.weight = weight
        else:
            self.weight = np.random.normal()
            
        self.dWeight = 0.0

In [38]:
class Neuron:
    alpha = 1

    def __init__(self, previousLayer, isOutput=False, weights=None):
        self.connections = [] #connections
        self.error = 0.0
        self.gradient = 0.0
        self.z = 0.0
        self.value = 0.0
        self.isOutput = isOutput
        if previousLayer:
            #conecta o neuronio com a camada anterior
            for neuron, index in zip(previousLayer, range(len(previousLayer))):
                if weights:
                    connection = Connection(neuron, weights[index])
                else:
                    connection = Connection(neuron)
                self.connections.append(connection)
        else:
            pass
    
    def addError(self, err):
        self.error = self.error + err

    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x * 1.0))

    def dSigmoid(self, x):
        return x * (1.0 - x)

    def setError(self, err):
        self.error = err

    def setValue(self, value):
        self.value = value

    def getValue(self):
        return self.value
    
    def feedForward(self):
        sumValue = 0
        if len(self.connections) == 0:
            print('a', self.value)
            return
        for connection in self.connections:
            sumValue = sumValue + connection.connectedNeuron.getValue() * connection.weight
        print('z:', round(sumValue,5))
        self.value = self.sigmoid(sumValue)
        print('a:', round(self.value,5))
        
    # δweight= η x gradient x output of connected neuron + α x previous δweight
    # trabalho atualiza de outra forma
    
    def backPropagate(self):
#         pdb.set_trace()
        #propagates it's error to the previous layer neurons
        if self.connections:
            for connection in self.connections:
                if connection.connectedNeuron.connections:                    
                    connection.connectedNeuron.error += self.error * connection.weight

            #calculate it's own error
            if not self.isOutput:
                self.error = self.error * self.value * (1 - self.value)
            print('Delta: ', round(self.error,5))
#             pdb.set_trace()
            for connection in self.connections:
                self.gradient = self.error * connection.connectedNeuron.value
                print('Gradient: ', round(self.gradient,5))
                connection.weight = connection.weight - (self.alpha * self.gradient)
#                 connection.weight = self.gradient * regularization
#                 print('Delta: ', round(self.error,5))

            self.error = 0

In [28]:
class Network:
    def __init__(self, topology, weights=None, regularization=1):
        self.layers = []
        self.regularization = regularization
        self.j = 0.0
        
        for numNeuron, index in zip(topology, range(len(topology))):
            layer = []
            layer.append(Neuron(None)) # bias neuron
            layer[-1].setValue(1) # setting output of bias neuron as 1
            if index == len(topology) -1:
                isOutput = True
            else:
                isOutput = False
            #
            for i in range(numNeuron):
                
                #input layer
                if (len(self.layers) == 0):
                    layer.append(Neuron(None))
                else:
                    #recebe layer de trás
                    if weights:
                        layerWeights = weights[index-1]
                        layer.append(Neuron(self.layers[-1], isOutput, layerWeights[i]))
                    else:
                        layer.append(Neuron(self.layers[-1], isOutput))
            
            self.layers.append(layer)

            
            
    def setInput(self, inputs):
        for i in range(len(inputs)):
            self.layers[0][i+1].setValue(inputs[i])
              

    def getError(self, target):
        error = 0
        
        if (type(target) != list):
            
            e = (target - self.layers[-1][1].getValue())
            error = error + e ** 2
        
        else:
            for i in range(len(target)):
                e = (target[i] - self.layers[-1][i].getValue())
                error = error + e ** 2
            
            error = error / len(target)
            
        error = math.sqrt(error)
        return error
        
    def jFunction(self, output, target):
        if (type(target) != list):
#             output = self.layers[-1][1].getValue()
            return - (target) * (np.log(output)) - (1 - target) * (np.log(1 - output))

        else:
            j = 0.0
            for i in range(len(target)):
#                 output = self.layers[-1][i+1].getValue()
                #j  +=  (target[i] * -1) * (np.log(output)) - (1 - target[i]) * (np.log(1 - output)) + (self.regularization / (2 * inputSize)) + self.squaredWeightsSum()
                j  +=  (target[i] * -1) * (np.log(output)) - (1 - target[i]) * (np.log(1 - output))
            return j
        
    
    def feedForward(self):
#         pdb.set_trace()
        for layer in self.layers:
            for neuron in layer:
                neuron.feedForward();
                
#     def backPropagate(self, target):
#         if (type(target) != list):
#             self.layers[-1][1].setError(self.layers[-1][1].getValue() - target)
#         else:
#             for i in range(len(target)):
#                 self.layers[-1][i+1].setError(self.layers[-1][i+1].getValue() - target[i])
            
#         for layer in self.layers[::-1]: #reverse the order
#             for neuron in layer:
#                 neuron.backPropagate(self.regularization)
    
    def backPropagate(self, target):
        
        if (type(target) != list):
#             pdb.set_trace()
            self.layers[-1][1].setError(self.layers[-1][1].getValue() - target)
            print('f(x): ',round(self.layers[-1][1].getValue(),5))
            print('target: ', target)
            j = self.jFunction(self.layers[-1][1].getValue(), target)
            print('j: ', round(j,3))
        else:
            j = 0.0
            for i in range(len(target)):
                # y - ftheta
#                 pdb.set_trace()
                self.layers[-1][i+1].setError(self.layers[-1][i+1].getValue() - target[i])
                j += self.jFunction(self.layers[-1][i+1].getValue(), target)
                print('f(x): ',round(self.layers[-1][i+1].getValue(),5))
                print('target: ', target[i])
            print('j: ', round((j/len(target)),3))
            
            
        for layer in self.layers[::-1]: #reverse the order
            for neuron in layer:
                neuron.backPropagate()
                
    def getResults(self):
        output = []
        for neuron in self.layers[-1]:
            
            #ignore bias neuron
            if not neuron.connections:
                continue
            output.append(neuron.getValue())
        return output

    def getThResults(self):
        output = []
        for neuron in self.layers[-1]:
            
            # ignore bias neuron
            if not neuron.connections:
                continue
            
            o = neuron.getValue()
            if (o > 0.5):
                o = 1
            else:
                o = 0
            output.append(o)
        #output.pop()# removing the bias neuron
        return output
    
    def train(self, inputs, outputs):
         
        error = 0
    
        for i in range(len(inputs)):
            self.setInput(inputs[i])
            self.feedForward()

            self.backPropagate(outputs[i])
            error = error + self.getError(outputs[i])
#             print ("error: ", error)
                
        
    

In [29]:
def normalize(data):
        return (data - min(data))/ (max(data) - min(data))

In [30]:
# targetColumn = 'target'
# dfWithoutTarget = df.loc[:, df.columns != targetColumn]
# outputs = df[targetColumn].values

# for feature in dfWithoutTarget.columns:
#     df[feature] = normalize(df[feature])

In [31]:
# inputs = df.loc[:, df.columns != targetColumn].values

In [32]:
x = round(0.6018070039467833, 5)
print(x)

0.60181


In [42]:
def main():
    topology = [2,4,3,2]
#     topology.append(8)
#     topology.append(8)
#     topology.append(1)


#     weights = [[[0.4, 0.1], [0.3, 0.2]], [[0.7, 0.5, 0.6]]]
    weights = [
        [[0.42000, 0.15000, 0.40000], [0.72000, 0.10000, 0.54000 ], [0.01000, 0.19000, 0.42000], [0.30000, 0.35000, 0.68000 ]], 
        [[0.21000, 0.67000, 0.14000, 0.96000, 0.87000],[0.87000, 0.42000, 0.20000, 0.32000, 0.89000],[0.03000, 0.56000, 0.80000, 0.69000, 0.09000]],
        [[0.04000, 0.87000, 0.42000, 0.53000],[0.17000, 0.10000, 0.95000, 0.69000]]
    ]
    net = Network(topology, weights)
    Neuron.eta = 0.09
    Neuron.alpha = 0.015
    
    inputs = [[0.83000, 0.02000]]
    outputs = [[0.75000, 0.28000]]
    net.train(inputs, outputs)
        
        

    print("end")
            
#     while True:
#         a = input("type 1st input :")
#         b = input("type 2nd input :")
#         net.setInput([a, b])
#         net.feedForward()
#         print (net.getThResults()) 


if __name__ == '__main__':
    main()

a 1
a 0.83
a 0.02
a 1
z: 0.5525
a: 0.63472
z: 0.8138
a: 0.69292
z: 0.1761
a: 0.54391
z: 0.6041
a: 0.64659
a 1
z: 1.81696
a: 0.8602
z: 2.02468
a: 0.88336
z: 1.37327
a: 0.79791
a 1
z: 1.58228
a: 0.82953
z: 1.64577
a: 0.83832
f(x):  0.82953
target:  0.75
f(x):  0.83832
target:  0.28
j:  1.929
Delta:  0.07953
Gradient:  0.07953
Gradient:  0.06841
Gradient:  0.07025
Gradient:  0.06346
Delta:  0.55832
Gradient:  0.55832
Gradient:  0.48027
Gradient:  0.4932
Gradient:  0.44549
Delta:  0.01503
Gradient:  0.01503
Gradient:  0.00954
Gradient:  0.01042
Gradient:  0.00818
Gradient:  0.00972
Delta:  0.05809
Gradient:  0.05809
Gradient:  0.03687
Gradient:  0.04025
Gradient:  0.0316
Gradient:  0.03756
Delta:  0.06892
Gradient:  0.06892
Gradient:  0.04374
Gradient:  0.04775
Gradient:  0.03748
Gradient:  0.04456
Delta:  0.12981
Gradient:  0.12981
Gradient:  0.10775
Gradient:  0.0026
Delta:  0.10047
Gradient:  0.10047
Gradient:  0.08339
Gradient:  0.00201
Delta:  0.14769
Gradient:  0.14769
Gradient:  0.1